In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from orders_helper import clean_order_data, get_delivery_data, summarise_delivery_data, aggregate_operator_line_data

%load_ext autoreload
%autoreload 2

# Data

## Raw Data 

In [ ]:
file_name = 'Online Operations Case Study_1.xlsx'

pick_activity_sheet = 'Picking Data'

raw_df = pd.read_excel(file_name, sheet_name=pick_activity_sheet)

In [ ]:
raw_df.head()

## Data Investigation

### General

In [ ]:
raw_df.isna().sum()

In [ ]:
raw_df.dtypes

In [ ]:
raw_df[raw_df.duplicated()]

In [ ]:
raw_df[raw_df['QTY'] == 0]

In [ ]:
raw_df['EVENT_TIME'].min(), raw_df['EVENT_TIME'].max()

In [ ]:
raw_df[raw_df['TOTE_NUMBER'] == raw_df['TOTE_CODE'].str[2:]]

In [ ]:
raw_df.groupby('ORDER_NUMBER').agg(**{
    'Order and Operator Distribution':('PICKER_ID','nunique')
    }).sort_values(by='Order and Operator Distribution').hist(bins=20)

In [ ]:
raw_df[(raw_df['ORDERED_QTY'] < raw_df['QTY']) & (raw_df['PICKED_UNIT_OF_MEASURE'] != 'GRAM')]

### Inspecting Pick Type

In [ ]:
raw_order_line_name = 'ORDER_PRODUCT_LINE_COUNT'

total_raw_orders_df = raw_df.groupby('PICK_TYPE').agg(**{
    raw_order_line_name:('PICKED_PRODUCT_ID','nunique'),
    'WEIGHT_INVESTIGATION':('PICK_WEIGHT','sum')})

total_raw_orders_df['WEIGHT_INVESTIGATION'] = total_raw_orders_df['WEIGHT_INVESTIGATION'] == 0

total_raw_orders_df.loc['TOTAL', raw_order_line_name] = total_raw_orders_df[raw_order_line_name].sum()

total_raw_orders_df

In [ ]:
raw_underpicked_df = raw_df[raw_df['QTY'] < raw_df['ORDERED_QTY']]
raw_underpicked_df.head()

In [ ]:
raw_underpicked_df.groupby('PICK_TYPE').agg({'PICKED_PRODUCT_ID':'nunique'})

In [ ]:
raw_df.groupby('PICK_TYPE')['PICKED_PRODUCT_ID'].count()

In [ ]:
raw_underpicked_df.groupby('PICK_TYPE')['PICKED_PRODUCT_ID'].count()

In [ ]:
raw_underpicked_df[raw_underpicked_df['PICK_TYPE'] == 'GNR']

In [ ]:
raw_df[(raw_df['QTY'] < raw_df['ORDERED_QTY']) & (raw_df['PICK_TYPE'] == 'GNR')]

In [ ]:
raw_df[(raw_df['QTY'] < raw_df['ORDERED_QTY']) & (raw_df['PICK_TYPE'] == 'NAG')]

In [ ]:
raw_df[(raw_df['ORDER_NUMBER'] == 246) & (raw_df['ORDERED_PRODUCT_ID'] == 6092)]

In [ ]:
raw_df[(raw_df['PICKED_UNIT_OF_MEASURE'] == 'GRAM') & (raw_df['PICK_TYPE'] != 'NORMAL')]

### Weight and Quantity

In [ ]:
raw_df[raw_df['PICK_WEIGHT'] == raw_df['PICK_WEIGHT'].max()]

In [ ]:
raw_df.loc[raw_df['PICK_TYPE'] == 'BULK', 'ORDERED_QTY'].min()

In [ ]:
raw_df.loc[raw_df['PICK_TYPE'] != 'BULK', :].sort_values(by='QTY', ascending=False)

In [ ]:
raw_df.loc[raw_df['PICK_TYPE'] != 'BULK', 'ORDERED_QTY'].max()

In [ ]:
raw_df[raw_df['ORDER_NUMBER'] == 424].head()

# Aggregations

In [ ]:
order_df = clean_order_data(raw_df)

In [ ]:
order_df[['ORDER_NUMBER', 'PICKED_PRODUCT_ID', 'PICKER_ID']].nunique()

In [ ]:
deliveries_df = get_delivery_data(order_df)
deliveries_df.head()

In [ ]:
deliveries_summary_df = summarise_delivery_data(deliveries_df)
deliveries_summary_df

In [ ]:
operator_info_df = order_df.groupby(['PICKER_ID','event_hour']).agg(**{
    'Total Orders':('ORDER_NUMBER','nunique'),
     'Total Units': ('FINAL_QTY','sum')}).reset_index()

operator_info_df

In [ ]:
operator_pack_df = aggregate_operator_line_data(operator_info_df)
operator_pack_df.head()

In [ ]:
operator_stats = operator_pack_df['Average Orders Per Hour'].describe()
operator_stats.loc['mean']

In [ ]:
operator_summary_df = operator_pack_df.groupby('Average Orders Per Hour Bin').agg(**{
    'Operator Count':('PICKER_ID','nunique')}).reset_index().sort_values(by='Average Orders Per Hour Bin', ascending=False).reset_index(drop=True)

operator_summary_df['Operator Count Cumsum'] = round(operator_summary_df['Operator Count'].cumsum() / operator_summary_df['Operator Count'].sum() * 100,2)

operator_summary_df

# Plots

Online Metrics

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

title = 'Delivery Packing Time Distribution'

y_label = 'Proportion (%)'

deliveries_summary_df.plot(y='Proportion Deliveries', kind='bar', ax=ax, ylabel=y_label)

plt.xticks(rotation=0)

plt.title(label = title, loc='left')

Store Colleagues Metric

In [ ]:
title = 'Store Colleagues Average Pick Speed Distribution'

x_label = 'Operator Bin Count'

ax = operator_summary_df.plot(kind='barh', x='Average Orders Per Hour Bin', y='Operator Count')

ax.xaxis.set_major_locator(MaxNLocator(integer=True))

ax.set_title(label=title, loc='left')

ax.set_xlabel(x_label)